<a href="https://colab.research.google.com/github/meti-94/TextClassification/blob/main/SingleStockTrading_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
## install finrl library
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git -q

     |████████████████████████████████| 234 kB 6.3 MB/s 
     |████████████████████████████████| 176 kB 42.9 MB/s 
     |████████████████████████████████| 59.6 MB 1.3 MB/s 
     |████████████████████████████████| 1.8 MB 39.3 MB/s 
     |████████████████████████████████| 124 kB 48.2 MB/s 
     |████████████████████████████████| 147 kB 46.0 MB/s 
     |████████████████████████████████| 45 kB 3.3 MB/s 
     |████████████████████████████████| 2.4 MB 37.9 MB/s 
     |████████████████████████████████| 153 kB 54.8 MB/s 
     |████████████████████████████████| 195 kB 52.4 MB/s 
     |████████████████████████████████| 90.8 MB 337 bytes/s 
     |████████████████████████████████| 448 kB 54.3 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 
     |████████████████████████████████| 1.1 MB 48.3 MB/s 
     |████████████████████████████████| 3.6 MB 32.2 MB/s 
     |████████████████████████████████| 271 kB 55.6 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████

In [3]:
from finrl.apps import config
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

In [4]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
from datetime import datetime

%matplotlib inline
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.finrl_meta.env_cryptocurrency_trading.env_btc_ccxt import BitcoinEnv
from finrl.drl_agents.stablebaselines3.models import DRLAgent
from finrl.finrl_meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [100]:
def read_data(path):
    hist = pd.read_csv(path, parse_dates=True)
    hist = hist[hist.Timestamp>=1388521800]
    hist['date'] = hist['Timestamp'].apply(lambda item:datetime.fromtimestamp(item))
    hist.set_index('date', inplace=True)
    hist.dropna(inplace=True)
    return hist

def resampling(hist, timeframe='1min'):
    ohlc = {
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume_(BTC)':'sum'
    }
    df = hist.resample(timeframe, origin='end_day').apply(ohlc)
    df.dropna(inplace=True)
    df.reset_index(inplace=True)
    df.rename(columns={'Open':'open', 'High':'high', 'Close':'close', 'Low':'low', 'Volume_(BTC)':'volume'}, inplace=True)
    df['date'] = df['date'].astype(str)
    df['tic'] = ['BTC' for _ in range(len(df))]
    df['day'] = [i%24 for i in range(len(df))]
    return df   
hist = read_data('/content/drive/MyDrive/kaggle_dataset/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv')
hist = hist['2019-01-01' :'2021-03-30']
hist = resampling(hist, timeframe='60min')
df = hist

In [101]:
df

,date,open,high,low,close,volume,tic,day
0,2019-01-01 00:00:00,3693.85,3698.79,3693.85,3698.00,5.491289,BTC,0
1,2019-01-01 01:00:00,3694.72,3703.25,3683.45,3694.36,93.774889,BTC,1
2,2019-01-01 02:00:00,3694.36,3699.35,3676.13,3681.18,98.683149,BTC,2
3,2019-01-01 03:00:00,3681.18,3683.55,3665.00,3680.84,106.559214,BTC,3
4,2019-01-01 04:00:00,3679.16,3686.73,3678.12,3682.12,183.172403,BTC,4
...,...,...,...,...,...,...,...,...
19672,2021-03-30 20:00:00,58851.42,59135.00,58733.00,59037.79,104.111136,BTC,16
19673,2021-03-30 21:00:00,59014.42,59032.73,58402.82,58591.06,125.871531,BTC,17
19674,2021-03-30 22:00:00,58592.54,58872.27,58570.57,58762.56,48.198945,BTC,18
19675,2021-03-30 23:00:00,58787.45,58941.34,58642.58,58657.09,67.727386,BTC,19


In [102]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
                    use_vix=False,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
Successfully added turbulence index


In [103]:
processed

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2019-01-01 00:00:00,3693.85,3698.79,3693.85,3698.00,5.491289,BTC,0,0.000000,3701.327737,3691.032263,0.000000,-66.666667,100.000000,3698.000000,3698.000000,0.000000
1,2019-01-01 01:00:00,3694.72,3703.25,3683.45,3694.36,93.774889,BTC,1,-0.081667,3701.327737,3691.032263,0.000000,-66.666667,100.000000,3696.180000,3696.180000,0.000000
2,2019-01-01 02:00:00,3694.36,3699.35,3676.13,3681.18,98.683149,BTC,2,-0.512771,3708.878859,3673.481141,0.000000,-100.000000,100.000000,3691.180000,3691.180000,0.000000
3,2019-01-01 03:00:00,3681.18,3683.55,3665.00,3680.84,106.559214,BTC,3,-0.708554,3706.364318,3670.825682,0.000000,-109.118506,100.000000,3688.595000,3688.595000,0.000000
4,2019-01-01 04:00:00,3679.16,3686.73,3678.12,3682.12,183.172403,BTC,4,-0.749089,3703.742384,3670.857616,7.436394,-46.755802,77.693647,3687.300000,3687.300000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19672,2021-03-30 20:00:00,58851.42,59135.00,58733.00,59037.79,104.111136,BTC,16,456.662881,59819.267061,56815.273939,61.408259,99.766095,27.036958,58101.457333,57089.263000,0.163110
19673,2021-03-30 21:00:00,59014.42,59032.73,58402.82,58591.06,125.871531,BTC,17,414.187317,59823.809943,56924.306057,57.901628,62.010524,13.702231,58116.505667,57126.520167,0.958498
19674,2021-03-30 22:00:00,58592.54,58872.27,58570.57,58762.56,48.198945,BTC,18,389.869569,59807.521063,57091.784937,58.835166,63.703513,13.702231,58156.877667,57170.390167,0.139002
19675,2021-03-30 23:00:00,58787.45,58941.34,58642.58,58657.09,67.727386,BTC,19,357.960683,59746.824941,57298.832059,58.016687,60.388228,15.814539,58192.812333,57213.845000,0.054222


In [104]:
list_ticker = processed["tic"].unique().tolist()
list_date = pd.to_datetime(processed['date']).astype(str).to_list()
combination = list(itertools.product(list_date,list_ticker))
processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])
processed_full = processed_full.fillna(0)

In [105]:
processed_full

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2019-01-01 00:00:00,BTC,3693.85,3698.79,3693.85,3698.00,5.491289,0,0.000000,3701.327737,3691.032263,0.000000,-66.666667,100.000000,3698.000000,3698.000000,0.000000
1,2019-01-01 01:00:00,BTC,3694.72,3703.25,3683.45,3694.36,93.774889,1,-0.081667,3701.327737,3691.032263,0.000000,-66.666667,100.000000,3696.180000,3696.180000,0.000000
2,2019-01-01 02:00:00,BTC,3694.36,3699.35,3676.13,3681.18,98.683149,2,-0.512771,3708.878859,3673.481141,0.000000,-100.000000,100.000000,3691.180000,3691.180000,0.000000
3,2019-01-01 03:00:00,BTC,3681.18,3683.55,3665.00,3680.84,106.559214,3,-0.708554,3706.364318,3670.825682,0.000000,-109.118506,100.000000,3688.595000,3688.595000,0.000000
4,2019-01-01 04:00:00,BTC,3679.16,3686.73,3678.12,3682.12,183.172403,4,-0.749089,3703.742384,3670.857616,7.436394,-46.755802,77.693647,3687.300000,3687.300000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19672,2021-03-30 20:00:00,BTC,58851.42,59135.00,58733.00,59037.79,104.111136,16,456.662881,59819.267061,56815.273939,61.408259,99.766095,27.036958,58101.457333,57089.263000,0.163110
19673,2021-03-30 21:00:00,BTC,59014.42,59032.73,58402.82,58591.06,125.871531,17,414.187317,59823.809943,56924.306057,57.901628,62.010524,13.702231,58116.505667,57126.520167,0.958498
19674,2021-03-30 22:00:00,BTC,58592.54,58872.27,58570.57,58762.56,48.198945,18,389.869569,59807.521063,57091.784937,58.835166,63.703513,13.702231,58156.877667,57170.390167,0.139002
19675,2021-03-30 23:00:00,BTC,58787.45,58941.34,58642.58,58657.09,67.727386,19,357.960683,59746.824941,57298.832059,58.016687,60.388228,15.814539,58192.812333,57213.845000,0.054222


In [106]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2019-01-01 00:00:00,BTC,3693.85,3698.79,3693.85,3698.00,5.491289,0,0.000000,3701.327737,3691.032263,0.000000,-66.666667,100.000000,3698.000000,3698.000000,0.0
1,2019-01-01 01:00:00,BTC,3694.72,3703.25,3683.45,3694.36,93.774889,1,-0.081667,3701.327737,3691.032263,0.000000,-66.666667,100.000000,3696.180000,3696.180000,0.0
2,2019-01-01 02:00:00,BTC,3694.36,3699.35,3676.13,3681.18,98.683149,2,-0.512771,3708.878859,3673.481141,0.000000,-100.000000,100.000000,3691.180000,3691.180000,0.0
3,2019-01-01 03:00:00,BTC,3681.18,3683.55,3665.00,3680.84,106.559214,3,-0.708554,3706.364318,3670.825682,0.000000,-109.118506,100.000000,3688.595000,3688.595000,0.0
4,2019-01-01 04:00:00,BTC,3679.16,3686.73,3678.12,3682.12,183.172403,4,-0.749089,3703.742384,3670.857616,7.436394,-46.755802,77.693647,3687.300000,3687.300000,0.0
5,2019-01-01 05:00:00,BTC,3682.22,3702.04,3679.63,3679.80,189.071570,5,-0.858580,3701.980524,3670.119476,6.526402,1.750707,12.893188,3686.050000,3686.050000,0.0
6,2019-01-01 06:00:00,BTC,3684.34,3692.85,3679.80,3690.85,113.479772,6,-0.413425,3701.724056,3671.747373,41.686081,9.654588,12.893188,3686.735714,3686.735714,0.0
7,2019-01-01 07:00:00,BTC,3690.85,3694.74,3686.11,3691.63,128.502042,7,-0.071191,3701.649055,3673.045945,43.244972,45.746091,7.369839,3687.347500,3687.347500,0.0
8,2019-01-01 08:00:00,BTC,3689.56,3716.79,3689.56,3700.44,228.584814,8,0.603979,3704.775684,3672.828761,56.753290,146.142567,33.317374,3688.802222,3688.802222,0.0
9,2019-01-01 09:00:00,BTC,3700.44,3725.58,3698.83,3715.09,186.813261,9,1.828751,3713.863570,3668.998430,69.316216,181.705254,42.588243,3691.431000,3691.431000,0.0


In [107]:
processed_full.sort_values(['date','tic'],ignore_index=True).tail(10)

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
19667,2021-03-30 15:00:00,BTC,58932.33,59039.61,58615.12,58910.90,128.390431,11,551.744505,59501.042940,56500.061060,61.303323,135.587261,28.079353,57967.518333,56872.005000,0.008427
19668,2021-03-30 16:00:00,BTC,58904.18,58921.10,58430.09,58661.86,209.161336,12,521.769272,59570.035304,56525.535696,59.468522,98.368261,21.325479,57990.972667,56913.885833,0.302037
19669,2021-03-30 17:00:00,BTC,58661.86,58841.28,58445.17,58717.03,96.402327,13,496.739340,59617.659692,56631.216308,59.744634,90.314149,21.325479,58012.405000,56953.794167,0.012902
19670,2021-03-30 18:00:00,BTC,58735.91,58885.00,58673.96,58747.94,61.725871,14,473.933903,59671.761990,56707.195010,59.902950,94.796664,22.429897,58041.872000,56994.350667,0.003513
19671,2021-03-30 19:00:00,BTC,58773.82,59144.94,58773.82,58849.28,198.557621,15,458.749541,59737.220028,56760.599972,60.430744,104.344750,28.783724,58066.568000,57042.739667,0.044748
19672,2021-03-30 20:00:00,BTC,58851.42,59135.00,58733.00,59037.79,104.111136,16,456.662881,59819.267061,56815.273939,61.408259,99.766095,27.036958,58101.457333,57089.263000,0.163110
19673,2021-03-30 21:00:00,BTC,59014.42,59032.73,58402.82,58591.06,125.871531,17,414.187317,59823.809943,56924.306057,57.901628,62.010524,13.702231,58116.505667,57126.520167,0.958498
19674,2021-03-30 22:00:00,BTC,58592.54,58872.27,58570.57,58762.56,48.198945,18,389.869569,59807.521063,57091.784937,58.835166,63.703513,13.702231,58156.877667,57170.390167,0.139002
19675,2021-03-30 23:00:00,BTC,58787.45,58941.34,58642.58,58657.09,67.727386,19,357.960683,59746.824941,57298.832059,58.016687,60.388228,15.814539,58192.812333,57213.845000,0.054222
19676,2021-03-31 00:00:00,BTC,58651.94,58770.38,58567.11,58760.59,78.047558,20,337.137930,59619.899090,57596.703910,58.601331,51.800440,12.595492,58221.545667,57264.520167,0.049842


In [108]:
# ['2020-01-01' :'2021-03-30']
train = data_split(processed_full, '2019-01-01','2020-06-30')
trade = data_split(processed_full, '2020-06-30','2021-03-30')
print(len(train))
print(len(trade))

13100
6552


In [109]:
train.tail()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
13095,2020-06-29 19:00:00,BTC,9163.89,9167.66,9129.07,9145.04,127.349874,15,9.584617,9163.876057,9040.814943,52.486581,112.916680,10.375720,9109.676000,9080.522333,0.316052
13096,2020-06-29 20:00:00,BTC,9146.40,9168.35,9127.16,9160.98,65.755434,16,12.901805,9171.002014,9037.863986,53.472453,120.139899,9.653320,9113.318000,9080.455333,0.190815
13097,2020-06-29 21:00:00,BTC,9162.08,9195.00,9144.07,9187.93,98.767524,17,17.503570,9184.586835,9031.916165,55.101848,170.815990,17.688448,9114.383667,9080.887667,0.525334
13098,2020-06-29 22:00:00,BTC,9187.93,9240.36,9187.93,9228.76,190.550189,18,24.166565,9205.078973,9020.049027,57.437903,253.462685,29.152649,9117.029000,9082.548167,1.185682
13099,2020-06-29 23:00:00,BTC,9233.06,9234.98,9178.34,9188.52,87.219592,19,25.901429,9214.665691,9017.534309,54.544510,192.857165,25.211380,9118.374333,9083.471500,1.095112


In [110]:
train.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2019-01-01 00:00:00,BTC,3693.85,3698.79,3693.85,3698.00,5.491289,0,0.000000,3701.327737,3691.032263,0.000000,-66.666667,100.000000,3698.000,3698.000,0.0
1,2019-01-01 01:00:00,BTC,3694.72,3703.25,3683.45,3694.36,93.774889,1,-0.081667,3701.327737,3691.032263,0.000000,-66.666667,100.000000,3696.180,3696.180,0.0
2,2019-01-01 02:00:00,BTC,3694.36,3699.35,3676.13,3681.18,98.683149,2,-0.512771,3708.878859,3673.481141,0.000000,-100.000000,100.000000,3691.180,3691.180,0.0
3,2019-01-01 03:00:00,BTC,3681.18,3683.55,3665.00,3680.84,106.559214,3,-0.708554,3706.364318,3670.825682,0.000000,-109.118506,100.000000,3688.595,3688.595,0.0
4,2019-01-01 04:00:00,BTC,3679.16,3686.73,3678.12,3682.12,183.172403,4,-0.749089,3703.742384,3670.857616,7.436394,-46.755802,77.693647,3687.300,3687.300,0.0


In [111]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.TECHNICAL_INDICATORS_LIST)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 1, State Space: 11


In [112]:
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [113]:
env_train, _ = e_train_gym.get_sb_env()

In [114]:
agent = DRLAgent(env = env_train)

In [115]:
model_a2c = agent.get_model("a2c")

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cuda device


In [116]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000)

-------------------------------------
| time/                 |           |
|    fps                | 214       |
|    iterations         | 100       |
|    time_elapsed       | 2         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -1.39     |
|    explained_variance | -0.151    |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | -0.164    |
|    reward             | -0.211434 |
|    std                | 0.969     |
|    value_loss         | 0.0268    |
-------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 214        |
|    iterations         | 200        |
|    time_elapsed       | 4          |
|    total_timesteps    | 1000       |
| train/                |            |
|    entropy_loss       | -1.39      |
|    explained_variance | 0.0695     |
|    learning_rate      | 0.0007     |
| 

In [117]:
# train = data_split(processed_full, '2019-01-01','2020-06-30')
# trade = data_split(processed_full, '2020-06-30','2021-03-30')
data_risk_indicator = processed_full[(processed_full.date<'2020-06-30') & (processed_full.date>='2019-01-01')]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [ ]:
insample_risk_indicator.vix.describe()

In [ ]:
insample_risk_indicator.vix.quantile(0.996)

In [118]:
insample_risk_indicator.turbulence.describe()

count    13100.000000
mean         1.402625
std         27.575383
min          0.000000
25%          0.028879
50%          0.159251
75%          0.585728
max       2893.306061
Name: turbulence, dtype: float64

In [119]:
insample_risk_indicator.turbulence.quantile(0.996)

29.9652728514055

In [120]:
e_trade_gym = StockTradingEnv(df=trade, turbulence_threshold=25, risk_indicator_col='turbulence', **env_kwargs)

In [121]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2020-06-30 00:00:00,BTC,9192.88,9197.72,9160.68,9187.53,33.165470,20,26.886505,9223.544100,9019.025900,54.474670,139.089416,18.115976,9119.518333,9084.038667,0.000055
1,2020-06-30 01:00:00,BTC,9191.53,9199.77,9166.46,9179.01,40.798992,21,26.672232,9230.095832,9020.070168,53.860692,131.676547,18.687739,9120.363667,9084.355500,0.042537
2,2020-06-30 02:00:00,BTC,9182.50,9201.95,9153.92,9199.97,74.248508,22,27.872420,9239.483812,9020.773188,55.147229,125.385668,13.505227,9123.076000,9085.306667,0.327109
3,2020-06-30 03:00:00,BTC,9199.72,9199.72,9169.80,9181.72,45.319960,23,27.039263,9242.306151,9030.447849,53.796100,110.213675,13.505227,9125.776000,9085.947833,0.214388
4,2020-06-30 04:00:00,BTC,9177.18,9183.40,9155.83,9161.71,30.236633,0,24.482125,9243.468705,9037.498295,52.341555,73.244835,7.528228,9127.563667,9086.407833,0.260911


In [122]:
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_a2c, 
    environment = e_trade_gym)

hit end!


In [123]:
df_account_value.shape

(6552, 2)

In [124]:
df_account_value.tail()

,date,account_value
6547,2021-03-29 19:00:00,5.006407e+06
6548,2021-03-29 20:00:00,4.994975e+06
6549,2021-03-29 21:00:00,4.949119e+06
6550,2021-03-29 22:00:00,4.971749e+06
6551,2021-03-29 23:00:00,4.990112e+06


In [125]:
df_actions.tail()

,date,actions
6546,2021-03-29 18:00:00,[0]
6547,2021-03-29 19:00:00,[0]
6548,2021-03-29 20:00:00,[0]
6549,2021-03-29 21:00:00,[0]
6550,2021-03-29 22:00:00,[0]


In [126]:
print("==============Get Backtest Results===========")
now = datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

==============Get Backtest Results===========
Annual return          0.063777
Cumulative returns     3.990112
Annual volatility      0.131440
Sharpe ratio           0.536003
Calmar ratio           0.216664
Stability              0.893009
Max drawdown          -0.294357
Omega ratio            1.113026
Sortino ratio          0.784151
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.054223
Daily value at risk   -0.016280
dtype: float64


In [127]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI", 
        start = df_account_value.loc[0,'date'],
        end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')


==============Get Baseline Stats===========


Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/multitasking/__init__.py", line 104, in _run_via_pool
    return callee(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/yfinance/multi.py", line 190, in _download_one_threaded
    timeout)
  File "/usr/local/lib/python3.7/dist-packages/yfinance/multi.py", line 207, in _download_one
    timeout=timeout)
  File "/usr/local/lib/python3.7/dist-packages/yfinance/base.py", line 153, in history
    _time.strptime(str(start), '%Y-%m-%d')))
  File "/usr/lib/python3.7/_strptime.py", line 571, in _strptime_time
    tt = _strptime(data_string, format)[0]
  File "/usr/lib/python3.7/_strptime.py", line 362, in _strptime
    data_string[found.end():])
ValueError: unconverted data remai

KeyboardInterrupt: ignored

In [128]:
print("==============Compare to DJIA===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = '^DJI', 
             baseline_start = df_account_value.loc[0,'date'],
             baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])

==============Compare to DJIA===========


Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/multitasking/__init__.py", line 104, in _run_via_pool
    return callee(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/yfinance/multi.py", line 190, in _download_one_threaded
    timeout)
  File "/usr/local/lib/python3.7/dist-packages/yfinance/multi.py", line 207, in _download_one
    timeout=timeout)
  File "/usr/local/lib/python3.7/dist-packages/yfinance/base.py", line 153, in history
    _time.strptime(str(start), '%Y-%m-%d')))
  File "/usr/lib/python3.7/_strptime.py", line 571, in _strptime_time
    tt = _strptime(data_string, format)[0]
  File "/usr/lib/python3.7/_strptime.py", line 362, in _strptime
    data_string[found.end():])
ValueError: unconverted data remai

KeyboardInterrupt: ignored